In [ ]:
import json
import pandas as pd
import re
import hashlib
import time
import os
from tqdm import tqdm
import re
import sys
from config import japanese_grammar

In [ ]:
import text_to_audio

In [ ]:
kanji_with_kana_pattern = re.compile(
    r"\s*(?P<kanji>[\u4E00-\u9FFF]) (?P<kana>[\u3040-\u309F]+)\s*"
)


def transform_content_to_ruby_rich_text(content, pattern=None):
    if pattern is None:
        pattern = kanji_with_kana_pattern

    return pattern.sub(r"<ruby>\1<rt>\2</rt></ruby>", content)


def split_kanji_and_its_kana(content, pattern=None):
    if pattern is None:
        pattern = kanji_with_kana_pattern

    return kanji_with_kana_pattern.sub(r"\1", content), kanji_with_kana_pattern.sub(
        r"\2", content
    )

In [ ]:
kanji_with_kana_pattern = re.compile(
    r"(?P<kanji>[\u4E00-\u9FFF])(?P<kana>[\u3040-\u309F]+)(?:\s|$)"
)
content = """
▶ あかい「赤い」 ②⓪（形）：红色的

解 古代日本人不区分“红”和“亮”两个概念，因为明亮的太阳光和火光都偏红色。

例 赤あか い花はな が咲さ いている。/红色的花开了。
"""

print(transform_content_to_ruby_rich_text(content, pattern=kanji_with_kana_pattern))

In [ ]:
kana_meaning_pattern = re.compile(r"^\d{2,}\s")
example_pattern = re.compile(r"^▶")

result = []
kanji_with_kana_pattern = re.compile(
    r"(?P<kanji>[\u4E00-\u9FFF]+)(?P<kana>[\u3040-\u309F]+)(?:\s|$)"
)
file_path = (
    japanese_grammar.book_folder
    / "记单词，一定要学的130个日语词根-_张铭_-_Z-Library_.txt"
)
with open(file_path, "r", encoding="utf-8") as fp:
    current_kana = ""
    current_meaning_item = ""
    current_meanning_explain = []
    current_example = ""
    current_example_explain = []
    within_example_part = False

    lines = fp.readlines()
    last_line = ""
    for line in lines:
        content = line.strip()
        if not content:
            continue

        if current_kana and content.startswith("第二章"):
            break

        # 假名
        if len(content) == 1:
            current_kana = content
            current_meaning_item = ""
            current_meanning_explain = []
            current_example = ""
            current_example_explain = []
            within_example_part = False
            continue

        # 假名词根含义
        if current_kana and kana_meaning_pattern.match(content):
            if current_example:
                result.append(
                    {
                        "kana": current_kana,
                        "meaning": current_meaning_item,
                        "explain": "<br>".join(current_meanning_explain),
                        "example": current_example,
                        "example_explain": "<br>".join(
                            [
                                transform_content_to_ruby_rich_text(
                                    content, pattern=kanji_with_kana_pattern
                                )
                                for content in current_example_explain
                            ]
                        ),
                    }
                )

            current_meaning_item = content
            current_meanning_explain = []
            current_example = ""
            within_example_part = False
            item_idx = content.split()[0]
            continue

        # 词根含义举例
        if current_kana and example_pattern.match(content):
            within_example_part = True
            if current_example:
                result.append(
                    {
                        "kana": current_kana,
                        "meaning": current_meaning_item,
                        "explain": "<br>".join(current_meanning_explain),
                        "example": current_example,
                        "example_explain": "<br>".join(
                            [
                                transform_content_to_ruby_rich_text(
                                    content, pattern=kanji_with_kana_pattern
                                )
                                for content in current_example_explain
                            ]
                        ),
                    }
                )

            current_example = content
            current_example_explain = []
            continue

        # 词根举例 解释
        if current_example:
            current_example_explain.append(content)
            continue

        # 假名词根 解释
        if current_kana and not within_example_part:
            current_meanning_explain.append(content)

        last_line = content


if current_example_explain:
    result.append(
        {
            "kana": current_kana,
            "meaning": current_meaning_item,
            "explain": "<br>".join(current_meanning_explain),
            "example": current_example,
            "example_explain": "<br>".join(
                [
                    transform_content_to_ruby_rich_text(
                        content, pattern=kanji_with_kana_pattern
                    )
                    for content in current_example_explain
                ]
            ),
        }
    )

In [ ]:
for record in result:
    if record["example"] == "▶ あか「垢」 ②（名）：污垢；水锈":
        print(record)
        break

In [ ]:
df = pd.DataFrame(result).fillna("")
print(df.shape)
df.to_csv("word_roots.csv", index=False)
df.to_csv("word_roots_anki.csv", index=False, header=False)
df.head()


In [ ]:
df.head()

In [ ]:
last_kana = ""
last_meaning = ""
for index in df.index:
    detail = df.loc[index].to_dict()
    if detail["kana"] != last_kana:
        print("=" * 32)
        print(detail["kana"])
    if detail["meaning"] != last_meaning:
        print()
        print(detail["meaning"])
        print(detail["explain"])

    example_explain = detail["example_explain"].split("<br>")[0]
    example_explain = "" if example_explain[0] != "解" else " ".join(example_explain.split()[1:])
    print(detail["example"])
    if example_explain:
        print(example_explain)

    last_kana = detail["kana"]
    last_meaning = detail["meaning"]

In [21]:
result_type_answer = []

kana_pattern = re.compile(r"(?<=^▶ )([\u3040-\u309F]+)(?=「)")
meaning_pattern = re.compile(r"(?<=：)(.+)$")
content_pattern = re.compile(r"^▶ ([\u3040-\u309F]+)(?:「(.+)」|\s?」)?\s*([⓪\u2460-\u2470]+)(（[·、\u4E00-\u9FFFサ]+）)：(.*)$")
"""
▶ あかい「赤い」 ②⓪（形）：红色的
▶ あける「明ける」 ⓪（自一）：天亮了；过年
▶ {{c1::あかい}}「赤い」 ②⓪（形）：{{c2::红色的}}
"""
for record in result:
    result_type_answer.append({
        "word": content_pattern.sub(r"{{c1::\1}}<br>\2 \3 \4<br>{{c2::\5}}", record["example"]),
        "word_explain": record["example_explain"],
        "kana_meaning": record["meaning"],
        "kana_meaning_explain": record["explain"],
    })

df_res = pd.DataFrame(result_type_answer, dtype=str).fillna("")
df_res.to_csv("word_roots(type_answer_anki).csv", index=False, header=False)
print(df_res.shape)
df_res.head()

(719, 4)


,word,word_explain,kana_meaning,kana_meaning_explain
0,{{c1::あかい}}<br>赤い ②⓪ （形）<br>{{c2::红色的}},解 古代日本人不区分“红”和“亮”两个概念，因为明亮的太阳光和火光都偏红色。<br>例 <r...,01 「あ」表 示“明亮”，以“あ＋か行假名”为代表,解析 あ是50 音图中发音最洪亮的假名，故有明亮之感。<br>熟词联想 あかい「赤い」：红色的
1,{{c1::あか}}<br>赤 ① （名）<br>{{c2::红色}},例 <ruby>好<rt>す</rt></ruby>きな<ruby>色<rt>いろ</rt>...,01 「あ」表 示“明亮”，以“あ＋か行假名”为代表,解析 あ是50 音图中发音最洪亮的假名，故有明亮之感。<br>熟词联想 あかい「赤い」：红色的
2,{{c1::あかるい}}<br>明るい ⓪③ （形）<br>{{c2::明亮的；明朗的}},解 あか→明亮。<br>例 <ruby>明<rt>あか</rt></ruby>るい<ruby...,01 「あ」表 示“明亮”，以“あ＋か行假名”为代表,解析 あ是50 音图中发音最洪亮的假名，故有明亮之感。<br>熟词联想 あかい「赤い」：红色的
3,{{c1::あきらか}}<br>明らか ② （形动）<br>{{c2::明朗的；清楚的}},解 あき→明亮，らか→形容动词后缀。<br>例 <ruby>犯人<rt>はんにん</rt><...,01 「あ」表 示“明亮”，以“あ＋か行假名”为代表,解析 あ是50 音图中发音最洪亮的假名，故有明亮之感。<br>熟词联想 あかい「赤い」：红色的
4,{{c1::あける}}<br>明ける ⓪ （自一）<br>{{c2::天亮了；过年}},解 あけ→明亮。<br>例 あけまして、おめでとうございます。/过年好。,01 「あ」表 示“明亮”，以“あ＋か行假名”为代表,解析 あ是50 音图中发音最洪亮的假名，故有明亮之感。<br>熟词联想 あかい「赤い」：红色的


In [ ]:
def get_md5(content):
    content = content.encode("utf-8")
    md5_hash = hashlib.md5()
    md5_hash.update(content)
    return md5_hash.hexdigest()


In [ ]:
import pandas as pd
import re

sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
groups = []
with open("CS_Word_20240405_22.25.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if matched == True and last_line[-1] != "。":
            sentence["content"] += line
            last_line = line
            continue

        if sentence_content_pattern.match(line.strip()):
            matched = True
            sentence = {"content": sentence_content_pattern.sub(r"\1 ", line)}
        else:
            if matched:
                sentence["hiragana"] = ""
                sentence["meaning"] = line
                if sentence["meaning"][1] == " ":
                    sentence["meaning"] = sentence["meaning"][2:]

                if sentence["meaning"][0] == "囉":
                    sentence["meaning"] = sentence["meaning"][1:]

                if sentence["content"][0] == "①" and groups:
                    example_sentences.append(groups)
                    groups = []

                groups.append(sentence)
            matched = False
        last_line = line

if groups:
    example_sentences.append(groups)

print(example_sentences[1])
with open("example_sentences.json", "w", encoding="utf-8") as fp:
    fp.write(json.dumps(example_sentences, ensure_ascii=False, indent=4))
    # pd.DataFrame(example_sentences, dtype=str).fillna("").to_json(fp, orient="records", force_ascii=False, indent=4)

In [ ]:
import json
import pandas as pd

df = pd.read_json("grammar_pd.json", dtype=str)
with open("grammar.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())
    df = pd.DataFrame(data)
df.head()
detail = df.loc[0].to_dict()
print(type(detail["example"]))
print(detail["example"])

In [ ]:
from grammar_enumeration import grammars, df_grammars

In [ ]:
df_grammars.shape

In [ ]:
df_grammars.hiragana.str.replace(r"[（）()\s()]", "").str.contains(r'なしに')[23]

In [ ]:
keyword = "なくして(は)"
df = df_grammars
df[df.content.str.replace(r"[（）()\s()]", "").str.contains(keyword)
                    | df.hiragana.str.replace(r"[（）()\s()]", "").str.contains(keyword)
                    | df.chinese_meaning.str.contains(keyword) | (df.source == keyword)]

In [ ]:
"其中N1级别201个，N2级别150个，N3级别140个，N4级别130个，N5级别120个。"

import json
import re

level_pattern = re.compile(r"^N([1-5])文法")
unit_pattern = re.compile(r"^第(\d+)单元(.*)")
grammar_start_pattern = re.compile(r"^\d+\. ")
seq_pattern = re.compile(r"^（\d+）")
from collections import defaultdict

lines = []
part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
level = 0
current_grammar = ""
current_part = ""
current_seq = ""
output = False
# with open("日语蓝宝书.txt", "r", encoding="utf-8") as fp:
# for idx, line in enumerate(fp.readlines()):
content = """



89. ～とて


（1）

接続

名＋とて

説明

表示假定条件的逆接，“即使……也不例外”“即使是……也……”“甚至……”“就连……”。

例文

△常に冷静な彼とて やはり人間だから、感情的になってしまうこともあるのだろう。 【2009年12月真题】/尽管他平时很冷静，但是也有情绪化的时候吧 。

△最近の電気製品は機能が多すぎる。開発者たちとて すべての機能が必要とは思わないのではないか。 【2007年真题】/最近的电器功能实在太多，就算是开发商也未必会认为所有的功能都有必要吧 。

△私とて 試合に負けたことに悔しい。 /输掉了比赛，我也很懊恼 。

注意

①前面主要接续人名。

②意思与「～としても」 相同，但「～とて」 是比较生硬的表达方式。

（2）

接続

名＋だ/動た形＋とて

説明

表示假定条件的逆接，无论前项怎样，后项的原则都是一样的。“即使……也……”“即便……也……”“就算……也……”。

例文

△たとえ病気だとて 試験に欠席してはいけない。 /就算生病了也不能缺考 。

△いくら 頼 たの んだとて 、できないことはできない。 /再怎么拜托，不能做的事情就是不能做 。

△どんなに後悔したとて 、過ぎたことは今さらどうしようもない。 /就算再怎么后悔，对于已经过去的事情也没有办法 。

注意

常与「たとえ」 「どんなに」 「いくら」 等词一起使用。




"""
grammars = {}
for _ in range(1):
    for idx, line in enumerate(content.split("\n")):
        line = line.strip()
        if not line:
            continue

        if res := level_pattern.findall(line):
            level = int(res[0])
            # print(f"Current Level: {level}")
            continue

        if res := unit_pattern.findall(line):
            # print(f"Current Unit: {res[0][0]}.{res[0][1]}")
            continue

        if grammar_start_pattern.match(line):
            print(f"Current Grammar: {line}")
            current_grammar = line
            grammars[current_grammar] = {}
            # current_part = ""
            # current_seq = ""
            continue

        if seq_pattern.match(line):
            # print(f"Current Seq: {line}")
            current_seq = line
            continue

        lines.append(line)

        if len(line) < 5 and line[:2] in part_keywords:
            current_part = line[:2]
            if current_part == "例文":
                grammars[current_grammar][current_part] = []
            else:
                grammars[current_grammar][current_part] = ""
            continue

        if current_part == "例文":
            grammars[current_grammar][current_part].append(line)
        else:
            grammars[current_grammar][current_part] += line + "\n"

print(json.dumps(grammars, ensure_ascii=False, indent=4))

In [ ]:
"其中N1级别201个，N2级别150个，N3级别140个，N4级别130个，N5级别120个。"

import json
import re

content = """



89. ～とて


（1）

接続

名＋とて

説明

表示假定条件的逆接，“即使……也不例外”“即使是……也……”“甚至……”“就连……”。

例文

△常に冷静な彼とて やはり人間だから、感情的になってしまうこともあるのだろう。 【2009年12月真题】/尽管他平时很冷静，但是也有情绪化的时候吧 。

△最近の電気製品は機能が多すぎる。開発者たちとて すべての機能が必要とは思わないのではないか。 【2007年真题】/最近的电器功能实在太多，就算是开发商也未必会认为所有的功能都有必要吧 。

△私とて 試合に負けたことに悔しい。 /输掉了比赛，我也很懊恼 。

注意

①前面主要接续人名。

②意思与「～としても」 相同，但「～とて」 是比较生硬的表达方式。

（2）

接続

名＋だ/動た形＋とて

説明

表示假定条件的逆接，无论前项怎样，后项的原则都是一样的。“即使……也……”“即便……也……”“就算……也……”。

例文

△たとえ病気だとて 試験に欠席してはいけない。 /就算生病了也不能缺考 。

△いくら 頼 たの んだとて 、できないことはできない。 /再怎么拜托，不能做的事情就是不能做 。

△どんなに後悔したとて 、過ぎたことは今さらどうしようもない。 /就算再怎么后悔，对于已经过去的事情也没有办法 。

注意

常与「たとえ」 「どんなに」 「いくら」 等词一起使用。




"""

level_pattern = re.compile(r"^N([1-5])文法")
unit_pattern = re.compile(r"^第(\d+)单元(.*)")
grammar_start_pattern = re.compile(r"^\d+\. ")
seq_pattern = re.compile(r"^（\d+）")


def get_japanese_sequence_sign(number):
    number = int(number)
    if number <= 0:
        return "⓪"

    return chr(ord("\u2460") + number - 1)


def extract_grammars(content):
    grammars = {}
    lines = []
    part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
    keyword_mapping = {
        '接続': "接续",
        '説明': "说明",
        '读法': "说明",
        '例词': "说明",
        '补充': "注意",
    }
    level = 0
    current_grammar = ""
    current_part = ""
    current_seq = ""
    for idx, line in enumerate(content.split("\n")):
        line = line.strip()
        if not line:
            continue

        line = line.replace("真题】", "JLPT】")
        if res := level_pattern.findall(line):
            level = int(res[0])
            # print(f"Current Level: {level}")
            continue

        if res := unit_pattern.findall(line):
            # print(f"Current Unit: {res[0][0]}.{res[0][1]}")
            continue

        if grammar_start_pattern.match(line):
            # print(f"Current Grammar: {line}")
            current_grammar = line
            grammars[current_grammar] = {}
            grammars[current_grammar]["level"] = level
            # current_part = ""
            # current_seq = ""
            continue

        if seq_pattern.match(line):
            print(f"Current Seq: {line}")
            current_seq = line
            continue

        lines.append(line)

        if len(line) < 5 and line[:2] in part_keywords:
            current_part = line[:2]
            current_part = keyword_mapping.get(current_part, current_part)
            index = f"{get_japanese_sequence_sign(line[2])} " if len(line) > 2 else ""
            if current_part not in grammars[current_grammar]:
                if current_part == "例文":
                    grammars[current_grammar][current_part] = []
                else:
                    grammars[current_grammar][current_part] = ""
            continue

        try:
            if current_part == "例文":
                grammars[current_grammar][current_part].append(line)
            else:
                grammars[current_grammar][current_part] += index + line + "\n"
                index = ""
        except Exception:
            print((idx, line))
            raise

    return grammars


In [ ]:
json.dumps("⓪①②③")

In [ ]:
chr(ord("\u2460"))

In [ ]:
kanji_with_kana_pattern = re.compile(r"(?P<kanji>[\u4E00-\u9FFF]) (?P<kana>[\u3040-\u309F]+) ")
content = "中 なか 川 がわ 先 せん 生 せい 「あ、 山 やま 口 ぐち さん。 偶 ぐう 然 ぜん ですね。」"
content = "辞书形 ⇒ 尊他语(辞书形)"
content = "做 | する ⇒ なさる"
content = "84. ～ 中 ちゅう/じゅう"
print(kanji_with_kana_pattern.findall(content))
kanji_with_kana_pattern.sub(r"<ruby>\1<rt>\2</rt></ruby>", content)

In [ ]:


content = "△お 客 きゃく 様 さま 、このお 皿 さら をさげてもよろしいでしょうか 。 /客人您好，这个盘子我可以撤下去了吗 ？"
print(transform_content_to_ruby_rich_text(content))


print(split_kanji_and_its_kana(content))

In [ ]:
def merge_example_sentences(sentences):
    new_sentences = []
    for sentence in sentences:
        if not sentence.startswith("△"):
            new_sentences[-1] += "\n" + sentence
        elif sentence.startswith("△B"):
            new_sentences[-1] += "\n  " + sentence[1:]
        else:
            new_sentences.append(sentence)

    return new_sentences


with open(japanese_grammar.book_folder / "日语蓝宝书.txt", "r", encoding="utf-8") as fp:
    content = fp.read()

empty_usage = []
grammars = extract_grammars(content)
# print(json.dumps(grammars, ensure_ascii=False, indent=4))
part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
example_sentence_pattern = re.compile(r"^([^/]+?)(【\d+年(?:\d+月)?JLPT】)?/(.*)$", re.DOTALL)
header_pattern = re.compile(r"^△\s*")
formatted_grammars = []
for idx, (grammar, detail) in enumerate(grammars.items()):
    try:
        content = grammar_start_pattern.sub("", grammar)
        hiragana = ""
        rich_text = transform_content_to_ruby_rich_text(content)
        if rich_text != content:
            content, hiragana = split_kanji_and_its_kana(content)

        example_sentences = detail.get("例文") or []
        example_sentences = merge_example_sentences(example_sentences)
        formated_exmpale_sentences = []

        for sentence_seq, sentence in enumerate(example_sentences):
            sentence_content, tag, meaning = example_sentence_pattern.findall(sentence)[0]
            sentence_content = header_pattern.sub("", sentence_content)
            sentence_content = f"{get_japanese_sequence_sign(sentence_seq+1)} {sentence_content}"

            sentence_content = transform_content_to_ruby_rich_text(sentence_content)
            formated_exmpale_sentences.append({"content": sentence_content, "tag": tag, "meaning": meaning})

        # example_sentences = [{"content": sentence} for sentence in map(transform_content_to_ruby_rich_text, detail.get("例文") or [])]

        formatted_grammars.append({
            "id": idx + 1,
            "content": content,
            "hiragana": hiragana,
            "meaning": detail.get("说明", "").strip(),
            "usage": transform_content_to_ruby_rich_text(detail.get("接续", "").strip()),
            "example": formated_exmpale_sentences,
            "remark": transform_content_to_ruby_rich_text(detail.get("注意", "").strip()),
            "source": "S2N1-N5",
            "japanese_meaning": "",
            "chinese_meaning": detail.get("说明", "").strip(),
            "level": detail["level"]
        })
        if formatted_grammars[-1]["usage"] and formatted_grammars[-1]["usage"] != transform_content_to_ruby_rich_text(formatted_grammars[-1]["usage"]):
            empty_usage.append(formatted_grammars[-1])
    except Exception:
        print(grammar)
        print(content)
        print(detail)
        raise

print(formatted_grammars[0])
with open("grammars_N1-N5.json", "w", encoding="utf-8") as fp:
    fp.write(json.dumps(formatted_grammars, ensure_ascii=False, indent=4))

In [ ]:
for detail in empty_usage:
    print(detail["usage"])
    print(transform_content_to_ruby_rich_text(detail["usage"]))
    print("=" * 32)

In [ ]:
# 日语平假名unicode编码范围：\u3040-\u309F
# 日语片假名unicode编码范围：\u30A0-\u30FF

In [ ]:
content = "私の料理を一口食べるなり 、父は変な顔をして席を立ってしまった。"
get_md5(content)

In [ ]:
import os
import shutil
from zipfile import ZipFile


def extract_epub_images(epub_path, output_folder):
    with ZipFile(epub_path, 'r') as epub_file:
        for file_name in epub_file.namelist():
            if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                output_file_path = os.path.join(output_folder, os.path.basename(file_name))
                with open(output_file_path, 'wb') as output_file:
                    output_file.write(epub_file.read(file_name))

    print("Image extraction completed.")


# 示例用法
epub_path = japanese_grammar.book_folder  / "超值白金版.蓝宝书大全集：新日本语能力考试N1-N5文法详解（最新修订版） (许小明) (Z-Library).epub" # 替换为实际的EPUB文件路径
output_folder = japanese_grammar.book_folder / "temp"  # 替换为实际的输出文件夹路径

extract_epub_images(epub_path, output_folder)

In [ ]:
from fpdf import FPDF
import os


def merge_jpg_to_pdf(folder_path, output_path):
    pdf = FPDF()

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".jpg"):
            file_path = os.path.join(folder_path, file_name)
            pdf.add_page()
            pdf.image(file_path, x=0, y=0, w=210)  # 设置图片大小为A4纸尺寸

    pdf.output(output_path, "F")
    print("PDF merging completed.")


# 示例用法
folder_path = japanese_grammar.book_folder / "temp"  # 替换为实际的输出文件夹路径
output_path = japanese_grammar.book_folder / "file.pdf"  # 替换为实际的输出文件夹路径

merge_jpg_to_pdf(folder_path, output_path)

In [ ]:
!python -m pip install fpdf

In [ ]:
import time
import os
from tqdm import tqdm
import re
import sys

kana_meaning_pattern = re.compile(r"^[△\u2460-\u2470\s]*")
rich_text_pattern = re.compile(r"<ruby>(.*?)<rt>.*?</rt></ruby>")
RE_WHITESPACES_PATTERN = re.compile(r"\s+")

with open("grammars_N1-N5.json", "r", encoding="utf-8") as fp:
    formatted_grammars = json.loads(fp.read())

total_sentence_amount = sum([len(grammar["example"]) for grammar in formatted_grammars])
print(f"Total sentence amount: {total_sentence_amount}")

for grammar in tqdm(formatted_grammars, file=sys.stdout):
    for sentence in grammar["example"]:
        content = sentence["content"]
        content = kana_meaning_pattern.sub("", content)
        content = rich_text_pattern.sub(r"\1", content)
        content = RE_WHITESPACES_PATTERN.sub("", content)

        content_md5 = get_md5(content)
        audio_path = f"japanese_grammar/audio/{content_md5}.mp3"

        text_to_audio.generate_audio([
            (None, content),
        ], audio_path, log=tqdm.write)


In [ ]:
grammar_content_pattern = re.compile(r"^\d? ?[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")

grammar_content_pattern.findall("~ 際 (に)")

In [ ]:
content = "ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"
content = re.sub("\s+", " ", content)
print(content)

question_pattern = re.compile(r"[（(](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[)）]")
print(question_pattern.findall(content))

In [ ]:
re.findall(r"(?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+", "a だけ b に限り")

In [ ]:
question_pattern = re.compile(r"[(（](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[）)]*\?")
print(question_pattern.findall("こちらの会議室をご利用になる際は、 受付で必要事項をご記入ください。"))
print(question_pattern.findall("ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"))

In [ ]:
import pandas as pd
import re

grammar_content_pattern = re.compile(r"^\d [~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")
sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
with open("CS_Word_20240403_23.41.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if grammar_content_pattern.match(line):
            print(line)